**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sat Jul 25 08:18:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
 CLASSIFIED.gdoc   'Getting started.pdf'   yolov3
'Colab Notebooks'   HTML.gdoc		   yolov4-tiny-new


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
# Change current directory to DARKNET!
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_training.cfg
# (x + 5) * 3 : x -> num of classes y -> filters

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo -e 'thumbs_up\nthumbs_down' > /content/obj.names
!echo -e 'classes= 2\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > /content/obj.data
!mkdir /content/obj

In [ ]:
# Download weights darknet model 53
# !wget https://pjreddie.com/media/files/darknet53.conv.74
! cp "/content/gdrive/My Drive/yolov3/darknet53.conv.74" "/content/darknet"

**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/01.jpg         
  inflating: data/obj/01.txt         
  inflating: data/obj/010.jpg        
  inflating: data/obj/010.txt        
  inflating: data/obj/011.jpg        
  inflating: data/obj/011.txt        
  inflating: data/obj/012.jpg        
  inflating: data/obj/012.txt        
  inflating: data/obj/013.jpg        
  inflating: data/obj/013.txt        
  inflating: data/obj/014.jpg        
  inflating: data/obj/014.txt        
  inflating: data/obj/015.jpg        
  inflating: data/obj/015.txt        
  inflating: data/obj/016.jpg        
  inflating: data/obj/016.txt        
  inflating: data/obj/017.jpg        
  inflating: data/obj/017.txt        
  inflating: data/obj/018.jpg        
  inflating: data/obj/018.txt        
  inflating: data/obj/019.jpg        
  inflating: data/obj/019.txt        
  inflating: data/obj/02.jpg         
  inflating: data/obj/02.txt         
  inflating: data/obj/020.jpg        
  inflating: 

In [ ]:
# NOT USED!

# # We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# # If the index is different from 0 then we're going to change it.
# import glob
# import os
# import re

# txt_file_paths = glob.glob(r"data/obj/*.txt")
# for i, file_path in enumerate(txt_file_paths):
#     # get image size
#     with open(file_path, "r") as f_o:
#         lines = f_o.readlines()

#         text_converted = []
#         for line in lines:
#             print(line)
#             numbers = re.findall("[0-9.]+", line)
#             print(numbers)
#             if numbers:

#                 # Define coordinates
#                 text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
#                 text_converted.append(text)
#                 print(i, file_path)
#                 print(text)
#         # Write file
#         with open(file_path, 'w') as fp:
#             for item in text_converted:
#                 fp.writelines("%s\n" % item)

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/033.jpg', 'data/obj/018.jpg', 'data/obj/032.jpg', 'data/obj/144.jpg', 'data/obj/036.jpg', 'data/obj/160.jpg', 'data/obj/045.jpg', 'data/obj/128.jpg', 'data/obj/166.jpg', 'data/obj/014.jpg', 'data/obj/048.jpg', 'data/obj/124.jpg', 'data/obj/138.jpg', 'data/obj/013.jpg', 'data/obj/026.jpg', 'data/obj/060.jpg', 'data/obj/01.jpg', 'data/obj/09.jpg', 'data/obj/016.jpg', 'data/obj/131.jpg', 'data/obj/148.jpg', 'data/obj/120.jpg', 'data/obj/134.jpg', 'data/obj/064.jpg', 'data/obj/11.jpg', 'data/obj/16.jpg', 'data/obj/151.jpg', 'data/obj/065.jpg', 'data/obj/015.jpg', 'data/obj/058.jpg', 'data/obj/117.jpg', 'data/obj/017.jpg', 'data/obj/155.jpg', 'data/obj/163.jpg', 'data/obj/052.jpg', 'data/obj/18.jpg', 'data/obj/111.jpg', 'data/obj/164.jpg', 'data/obj/044.jpg', 'data/obj/145.jpg', 'data/obj/130.jpg', 'data/obj/113.jpg', 'data/obj/056.jpg', 'data/obj/162.jpg', 'data/obj/119.jpg', 'data/obj/068.jpg', 'data/obj/114.jpg', 'data/obj/110.jpg', 'data/obj/157.jpg', 'data/obj/035.jpg', 'dat

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train /content/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, out